Food Recipe Recomender system

Data exploration

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


https://realpython.com/sentiment-analysis-python/

In [2]:
import pandas as pd
import nltk
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tag import pos_tag
from nltk.corpus import stopwords
import re
import spacy

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [134]:
interactions_data = pd.read_csv("/content/gdrive/MyDrive/Information Retreival/RAW_interactions.csv")
recipes_data = pd.read_csv("/content/gdrive/MyDrive/Information Retreival/RAW_recipes.csv")

In [4]:
interactions_data.dtypes

user_id       int64
recipe_id     int64
date         object
rating        int64
review       object
dtype: object

In [5]:
recipes_data.dtypes

name              object
id                 int64
minutes            int64
contributor_id     int64
submitted         object
tags              object
nutrition         object
n_steps            int64
steps             object
description       object
ingredients       object
n_ingredients      int64
dtype: object

In [6]:
interactions_data.head()

,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."


In [135]:
recipes_data.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


Creating an indexer based on ingredients

In [136]:
recipe_data1 = recipes_data.loc[:,['id']]

In [137]:
recipe_data1.head()

,id
0,137739
1,31490
2,112140
3,59389
4,44061


Word tokenizing

In [138]:
recipe_data1['ingredients'] = recipes_data['ingredients'].apply(lambda x: re.sub('[\[\]\']+',' ',x).split(','))
recipe_data1['ingredients'] = recipe_data1['ingredients'].apply(lambda x: [i.strip() for i in x])

In [139]:
recipe_data1['ingredients_tokenized'] = recipes_data['ingredients'].apply(lambda x: re.sub('\W+',' ',x))
recipe_data1['ingredients_tokenized'] = recipe_data1['ingredients_tokenized'].apply(lambda x: nltk.tokenize.word_tokenize(x))
stop_words = set(stopwords.words('english'))
recipe_data1['ingredients_tokenized'] = recipe_data1['ingredients_tokenized'].apply(lambda x: [y for y in x if y not in stop_words])

In [140]:
recipe_data1

,id,ingredients,ingredients_tokenized
0,137739,"[winter squash, mexican seasoning, mixed spice...","[winter, squash, mexican, seasoning, mixed, sp..."
1,31490,"[prepared pizza crust, sausage patty, eggs, mi...","[prepared, pizza, crust, sausage, patty, eggs,..."
2,112140,"[ground beef, yellow onions, diced tomatoes, t...","[ground, beef, yellow, onions, diced, tomatoes..."
3,59389,"[spreadable cheese with garlic and herbs, new ...","[spreadable, cheese, garlic, herbs, new, potat..."
4,44061,"[tomato juice, apple cider vinegar, sugar, sal...","[tomato, juice, apple, cider, vinegar, sugar, ..."
...,...,...,...
231632,486161,"[celery, onion, green sweet pepper, garlic clo...","[celery, onion, green, sweet, pepper, garlic, ..."
231633,493372,"[paprika, salt, garlic powder, onion powder, d...","[paprika, salt, garlic, powder, onion, powder,..."
231634,308080,"[hard-cooked eggs, mayonnaise, dijon mustard, ...","[hard, cooked, eggs, mayonnaise, dijon, mustar..."
231635,298512,"[butter, eagle brand condensed milk, light bro...","[butter, eagle, brand, condensed, milk, light,..."


In [141]:
ingredients_unique = recipe_data1['ingredients'].explode().unique()
ingredients_unique1 = recipe_data1['ingredients_tokenized'].explode().unique()

In [142]:
ingredients_unique

array(['winter squash', 'mexican seasoning', 'mixed spice', ...,
       '"hunt s fire roasted diced tomatoes"', 'skinless halibut fillet',
       'nepitella'], dtype=object)

In [143]:
ingredient_indexer = {x:[] for x in ingredients_unique}
tokenized_indexer =  {x:[] for x in ingredients_unique1}

In [144]:
print(ingredient_indexer.keys())
print(tokenized_indexer.keys())

dict_keys(['winter squash', 'mexican seasoning', 'mixed spice', 'honey', 'butter', 'olive oil', 'salt', 'prepared pizza crust', 'sausage patty', 'eggs', 'milk', 'salt and pepper', 'cheese', 'ground beef', 'yellow onions', 'diced tomatoes', 'tomato paste', 'tomato soup', 'rotel tomatoes', 'kidney beans', 'water', 'chili powder', 'ground cumin', 'lettuce', 'cheddar cheese', 'spreadable cheese with garlic and herbs', 'new potatoes', 'shallots', 'parsley', 'tarragon', 'red wine vinegar', 'pepper', 'red bell pepper', 'yellow bell pepper', 'tomato juice', 'apple cider vinegar', 'sugar', 'clove oil', 'cinnamon oil', 'dry mustard', 'vanilla ice cream', 'frozen apple juice concentrate', 'apple', 'fennel seeds', 'green olives', 'ripe olives', 'garlic', 'peppercorn', 'orange rind', 'orange juice', 'red chile', 'extra virgin olive oil', 'pork spareribs', 'soy sauce', 'fresh garlic', 'fresh ginger', 'fresh coarse ground black pepper', 'fresh cilantro leaves', 'tomato sauce', 'brown sugar', 'yellow 

In [145]:
for i,d in recipe_data1.iterrows():
  for ing in d['ingredients']:
    ingredient_indexer[ing].append(d['id']) 

In [146]:
for i,d in recipe_data1.iterrows():
  for ing in d['ingredients_tokenized']:
    tokenized_indexer[ing].append(d['id']) 

In [147]:
print(ingredient_indexer['cheese'])

[31490, 43026, 33606, 59632, 522861, 457136, 455351, 151143, 392158, 176883, 440388, 370567, 216873, 533190, 113197, 79046, 221166, 360881, 182904, 464692, 31683, 394191, 9518, 133684, 99798, 75650, 513291, 35818, 147274, 15216, 158685, 274746, 120383, 140171, 11156, 499804, 309179, 411729, 43843, 231245, 102687, 344050, 376731, 66603, 366306, 532656, 274926, 392112, 455187, 80165, 54264, 98694, 411114, 493780, 473660, 216335, 110017, 364330, 226798, 82984, 338713, 414547, 455028, 230828, 180120, 395016, 47000, 247103, 225907, 364762, 83666, 110887, 161872, 24357, 242255, 412424, 476522, 419499, 87502, 83893, 309645, 57786, 446192, 18734, 141423, 105924, 455452, 457823, 44991, 321213, 105454, 256584, 340314, 481345, 483327, 137593, 13036, 322412, 13033, 235937, 75608, 71797, 63338, 383548, 330036, 163506, 187347, 382236, 384590, 170709, 236646, 114022, 359855, 42232, 335205, 388503, 324599, 8805, 370408, 173478, 206057, 467077, 484053, 142253, 523334, 92096, 116046, 155712, 269186, 400

In [148]:
ingredient_indexer2 = pd.DataFrame({'ingredients': list(ingredient_indexer.keys()), 'recipe_id': list(ingredient_indexer.values())})
tokenized_indexer2 = pd.DataFrame({'ingredients': list(tokenized_indexer.keys()), 'recipe_id': list(tokenized_indexer.values())})

Dataframe to save indexed values

In [149]:
ingredient_indexer2

,ingredients,recipe_id
0,winter squash,"[137739, 85912, 121604, 70514, 147409, 272858,..."
1,mexican seasoning,"[137739, 37595, 340923, 345720, 358881, 26117,..."
2,mixed spice,"[137739, 423369, 412761, 164295, 176727, 41771..."
3,honey,"[137739, 67888, 90921, 93959, 108414, 84797, 7..."
4,butter,"[137739, 42198, 107699, 49262, 27087, 93249, 9..."
...,...,...
14901,apricot jell-o,[20948]
14902,rosarita premium whole black beans,[485697]
14903,"""hunt s fire roasted diced tomatoes""",[485697]
14904,skinless halibut fillet,[334581]


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


In [150]:
tokenized_indexer2

,ingredients,recipe_id
0,winter,"[137739, 85912, 121604, 70514, 147409, 272858,..."
1,squash,"[137739, 31835, 112984, 130819, 223349, 167554..."
2,mexican,"[137739, 501028, 190108, 39893, 299989, 128715..."
3,seasoning,"[137739, 23850, 83873, 33606, 103948, 42522, 3..."
4,mixed,"[137739, 80050, 243251, 221116, 356803, 400273..."
...,...,...
4200,namak,[221218]
4201,stirrings,[423088]
4202,mixer,[423088]
4203,rosarita,[485697]


In [151]:
ingredient_indexer2.to_csv('/content/gdrive/MyDrive/ingredients_indexed_table.csv')
tokenized_indexer2.to_csv('/content/gdrive/MyDrive/tokenized_indexed_table.csv')

User input for query

In [152]:
query = input('Ingredients:').split(',')

Ingredients:cheese, olive oil, eggs


In [153]:
print(query)

['cheese', ' olive oil', ' eggs']


In [180]:
def query_preprocess(query):
  query_ingredients=[]
  query_tokens=[]
  for word in query:
    query_ingredients.append(word.strip())
    word = re.sub(r'\W+', ' ', word)
    query_tokens.extend(nltk.tokenize.word_tokenize(word.lower().strip()))
  return (query_ingredients, query_tokens)
query1, query2 = query_preprocess(query)

In [187]:
print(query1)
print(query2)

['cheese', 'olive oil', 'eggs']
['cheese', 'olive', 'oil', 'eggs']


In [181]:
def fetch_candidates(query1, query2):
  candidates_ingredients = []
  candidates_tokens = []
  for token in query1:
    setter = set(ingredient_indexer[token])
    candidates_ingredients.append(setter)
  for token in query2:
    setter = set(tokenized_indexer[token])
    candidates_tokens.append(setter)
  candidates1 = list(set.intersection(*map(set,candidates_ingredients)))  
  candidates2 = list(set.intersection(*map(set,candidates_tokens)))
  return (candidates1, candidates2)

candidate_list1, candidate_list2 = fetch_candidates(query1, query2)  


In [182]:
print(len(candidate_list1))
print(candidate_list1)
print(len(candidate_list2))
print(candidate_list2)

55
[290048, 219137, 396548, 311434, 41356, 104206, 471316, 70037, 380438, 353175, 387864, 287769, 245396, 472728, 102687, 438432, 52000, 234402, 437923, 405282, 354212, 290214, 455469, 330036, 455476, 437557, 323125, 74040, 464692, 59452, 112834, 51138, 175301, 523334, 383175, 30277, 528843, 316235, 502733, 389713, 159955, 440275, 264533, 171477, 502617, 356321, 113508, 328424, 126188, 483694, 371951, 407668, 313081, 446715, 24446]
1368
[122888, 376846, 114717, 262197, 491574, 139322, 114747, 229436, 98372, 311370, 49229, 368725, 491611, 483425, 237668, 245868, 82034, 131194, 475258, 303230, 303236, 360583, 311434, 450700, 32915, 270491, 245918, 82084, 8364, 114863, 499890, 229565, 344261, 286920, 73932, 90316, 368848, 73937, 245979, 377061, 270566, 319721, 516332, 8448, 360713, 311562, 82191, 319774, 74040, 475454, 295231, 278857, 401741, 278862, 57680, 8534, 24918, 24925, 328046, 483694, 287092, 147832, 196985, 278909, 475527, 188811, 41356, 131477, 41368, 65945, 414, 221599, 8608, 4

In [200]:
print(len(list(recipe_data1.loc[recipe_data1['id']==290048]['ingredients'])[0]))

10


Check if retrieved candidates have the tokens

In [210]:
relevance1 = []
for id in candidate_list1:
  if len(query1) < (len(list(recipe_data1.loc[recipe_data1['id']==id]['ingredients'])[0])):
    relevance1.append(0)
  else:
    for ingredient in query1:
      if ingredient in list(recipe_data1.loc[recipe_data1['id']==id]['ingredients'])[0]:
        relevance1.append(1)
      else:
        relevance1.append(0)

In [202]:
print(len(candidate_list1))
print(candidate_list1)

27
[219137, 311434, 104206, 70037, 353175, 287769, 472728, 438432, 234402, 405282, 290214, 330036, 437557, 74040, 59452, 51138, 523334, 30277, 316235, 389713, 440275, 171477, 356321, 328424, 483694, 407668, 446715]


In [211]:
relevance2 = []
for id in candidate_list2:
  if len(query2) < (len(list(recipe_data1.loc[recipe_data1['id']==id]['ingredients_tokenized'])[0])):
    relevance2.append(0)
  else:
    for ingredient in query2:
      if ingredient in list(recipe_data1.loc[recipe_data1['id']==id]['ingredients_tokenized'])[0]:
        relevance2.append(1)
      else:
        relevance2.append(0)

In [205]:
print(len(candidate_list2))
print(candidate_list2)

684
[376846, 262197, 139322, 229436, 311370, 368725, 483425, 245868, 131194, 303230, 360583, 450700, 270491, 82084, 114863, 229565, 286920, 90316, 73937, 377061, 319721, 8448, 311562, 319774, 475454, 278857, 278862, 8534, 24925, 483694, 147832, 278909, 188811, 131477, 65945, 221599, 426401, 410035, 461, 262622, 180710, 254445, 385520, 393716, 66039, 328201, 475666, 418324, 33312, 320061, 475723, 418393, 320097, 180849, 311933, 98947, 524933, 475785, 74387, 74401, 25253, 49839, 164546, 279250, 58084, 131837, 254738, 213788, 213798, 394036, 90947, 320342, 295783, 295791, 131956, 90998, 336768, 131978, 115611, 82862, 246729, 426965, 279515, 492516, 386035, 287769, 58394, 230436, 58417, 361531, 271422, 140358, 402514, 189522, 205918, 173161, 369786, 74891, 58529, 222392, 427211, 263379, 99540, 369882, 99565, 222460, 247053, 17688, 312604, 476467, 222524, 386368, 427332, 238929, 410963, 173410, 345446, 25974, 411027, 370085, 189870, 116149, 107989, 312791, 337415, 345610, 34326, 460321, 443

In [209]:
recipes_data.loc[recipes_data['id']==353175]['ingredients']

113113    ['olive oil', 'hot italian sausage', 'garlic',...
Name: ingredients, dtype: object

In [212]:
print(relevance1)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
